In [8]:
import glob
import itertools
from collections import OrderedDict
from random import shuffle

from shapely.geometry import LineString, mapping, shape, MultiPolygon
from shapely.ops import polygonize, unary_union
import fiona

In [9]:
polygonfilename = 'annual.geojson' #input polygons
fieldname_of_s1 = 'filename' #field with S1 filename associated with polygon

# summary of script / introduction to user variables:

# 1. estimation of all the fields' centroids
# 2. form buffer around these centroids

BUFFERSIZE_DEG = 0.18

#3. check for intersections of these buffers and neighboring centroids; this corresponds to file lists as each buffer/centroid is associated with an acquisition
#4. keep only supersets of the filelists to reduce computational demand
#5. take a superset (e.g. 30 S1-scenes with each month present), and build all subsets of 12 scenes which cover every month
#6. Estimate the size of overlap for SUBSET_PCT % of these files (greatly increases speed)

SUBSET_PCT = 0.05

#7. create geojsons for best subsets

schm = {'geometry': 'MultiPoint','properties': {fieldname_of_s1: 'str'}}

In [10]:
### recursive function to split a list into sublists, i.e.:
### purpose: [1,2a,2b,3] => [1,2a,3], [1,2b,3], just with S1 filenames in list concerning months
### very quick, but:
### fix input and return (right now depends on global variables:
### "fulllist, p, monthlies")

def recklist(liste=[], index=0):  
    if index < len(fulllist):        
        occurences = fulllist.count(fulllist[index])
        for o in range(occurences):
            newlist = liste[:]
            newlist.append(p[index + o])
            cont_here = len(fulllist) - fulllist[::-1].index(fulllist[index])
            recklist(newlist[:], cont_here)            
    else:
        return monthlies.append(liste)

In [56]:
with fiona.open(polygonfilename) as layer:
    all_centroid_shapes = []
    all_buffer_shapes = []
    all_properties = []
    for p in layer:
        all_centroid_shapes.append(shape(p['geometry']).centroid)
        all_buffer_shapes.append(shape(p['geometry']).centroid.buffer(BUFFERSIZE_DEG))
        all_properties.append({fieldname_of_s1: p['properties'][fieldname_of_s1]})
        
    # intersections is a dictionary
    # each entry is the index of an element in centroids, combined with filenames
    # of the scenes of the buffers it intersects..
    
    intersections = {}
    
    for j,c in enumerate(all_centroid_shapes):
        intersections_sub = []
        for i,b in enumerate(all_buffer_shapes):               
            if c.intersects(b):
                intersections_sub.append(all_properties[i][fieldname_of_s1])        
        covered_months = []
        for e in intersections_sub:
            covered_months.append(e[21:23])
        if len(set(covered_months)) == 12:
            intersections[j] = set(intersections_sub)
            
    od = OrderedDict(sorted(intersections.items(), key=lambda t: t[0]))
    
  
    maxid = od.keys()[-1]
    while i != maxid:
        hier = False
        for i in od:
            for j in od:
                if od[i] <= od[j] and i != j:
                    hier = i
                    break
            if hier == i:
                break
        if hier:
            del(od[hier])
    
    best_stacks = []
    best_areas = []
    
    for i in od:
        p = list(od[i])
        p.sort()
        fulllist = [e[19:23] for e in p]
        monthlies = []
        # this generates all potential stacks of monthly
        # coverage from a stack >= 12        
        recklist()
        
        best_area = 0
        
        shuffle(monthlies)
        
        for j, m in enumerate(monthlies[:int(SUBSET_PCT*len(monthlies))]):
            fin_shape = shape(filter(lambda f: f['properties'][fieldname_of_s1]==m[0], layer)[0]['geometry'])
            for n in m[1:]:      
                new_shape = shape(filter(lambda f: f['properties'][fieldname_of_s1]==n, layer)[0]['geometry'])
                fin_shape = fin_shape.intersection(new_shape)
            
            area = fin_shape.area
            if area > best_area:
                best_stack = m
                best_area = area
        
        best_stacks.append(best_stack)
        best_areas.append(best_area)
        
    
    i=0     
    for a, supset in enumerate(best_stacks):
        with fiona.open('suggestion_'+str(i)+'_witharea_'+str(best_areas[a])+'.geojson', 'w', 'GeoJSON', schm) as tgt: 
            for swath_outline in supset:
                fin_shape = shape(filter(lambda f: f['properties'][fieldname_of_s1]==swath_outline, layer)[0]['geometry'])
                tgt.write({'geometry': mapping(fin_shape), 'properties': {fieldname_of_s1:str(swath_outline)}})
        i+=1